# **HEP analysis in Python**

Previously in this course you've learnt about different ML techniques and how can they be used in HEP analysis. Now the question is how can we integrate all this ML stuff into typical analysis workflow. Today we will disscuss one possible answer - doing the entire analysis with python using __[scikit-hep](https://scikit-hep.org/)__

NB: This seminar's aim is to only provide some introduction into the scikit-hep ecosystem and it by no means provides a complete picture, for more information on topics discussed below you can follow links that will be provided along the way.

This seminar is in large part based on the materials from __[PyHEP 2020 workshop](https://indico.cern.ch/event/882824/timetable/#20200713.detailed)__

## **I/O with .root files**

### **PyROOT - dynamic bindings to ROOT**

One way to work with root files in python many of may be already familiar with is to use PyROOT - dynamic python bindings to ROOT framework and C++. 

In [ ]:
import ROOT
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

ROOT 6.22, a major revision of PyROOT has been released. The new PyROOT has extensive support for modern C++ (it operates on top of __[cppyy](https://cppyy.readthedocs.io)__)

Here is an example of how you can access C++ objects from python using this functionality

In [ ]:
v1 = ROOT.std.vector['float']((1, 2, 3))
print("ROOT.std.vector['float']", v1)

v2 = np.asarray(v1)
print('numpy.array', v2)

v1[0] = 42
print('numpy.array', v2)

In [ ]:
v1 = np.array((1, 2, 3), dtype=np.float32)
print('numpy.array', v1)

v2 = ROOT.VecOps.AsRVec(v1)
print("ROOT.RVec['float']", v2)

v1[0] = 42
print("ROOT.RVec['float']", v2)

We can call usuall  C++ commands under Pyhton enviroment:

In [ ]:
ROOT.gInterpreter.Declare('''
float get_element(float* v, unsigned int i) {
    return v[i];
}
''')

print('The first element of the numpy.array is', ROOT.get_element(v1, 0))

With PyROOT you can use usual ROOT classes as in following example:

In [ ]:
#path = 'root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root'
path = './data/opendata_muons_skimmed.root'
file = ROOT.TFile.Open(path)
tree = file.Events
tree.Print()
# Can iterate over t
dimuon_pt = []
for i,event in enumerate(tree):
    #if i % 1000 == 0:
    #    print('Processing event {}'.format(i))
    if event.nMuon == 2:
        dimuon_pt.append(event.Muon_pt[0] + event.Muon_pt[1])
    if i == 100000:
        break
plt.hist(dimuon_pt, range=(0, 100), bins=20);

### **RDataFrame**

The modern interface to process datasets in ROOT files (aka TTrees) is [RDataFrame](https://root.cern/doc/master/classROOT_1_1RDataFrame.html). The concept is a computation graph, which is built in a declarative manner, and executes the booked computations as efficient as possible.

In [ ]:
df = ROOT.RDataFrame('Events', path)

We filter the dataset for events with two muons and opposite charge. The last line restricts the full dataset to a subset of the in total 66 mio. events.

In [ ]:
df = df.Filter("nMuon == 2", "Events with exactly two muons")\
       .Filter("Muon_charge[0] != Muon_charge[1]", "Muons with opposite charge")\
       .Range(20000)

In [ ]:
ROOT.gInterpreter.Declare(
"""
using Vec_t = const ROOT::VecOps::RVec<float>&;
float compute_mass(Vec_t pt, Vec_t eta, Vec_t phi, Vec_t mass) {
    ROOT::Math::PtEtaPhiMVector p1(pt[0], eta[0], phi[0], mass[0]);
    ROOT::Math::PtEtaPhiMVector p2(pt[1], eta[1], phi[1], mass[1]);
    return (p1 + p2).mass();
}
""")
df = df.Define("Dimuon_mass", "compute_mass(Muon_pt, Muon_eta, Muon_phi, Muon_mass)")

In [ ]:
hist = df.Histo1D(("hist", ";m_{#mu#mu} (GeV);N_{Events}", 5000, 2, 200), "Dimuon_mass")

In [ ]:
report = df.Report()

In [ ]:
ROOT.gStyle.SetOptStat(0); ROOT.gStyle.SetTextFont(42)
c = ROOT.TCanvas("c", "", 800, 700)
c.SetLogx(); c.SetLogy()
hist.Draw()

label = ROOT.TLatex(); label.SetNDC(True)
label.SetTextSize(0.040); label.DrawLatex(0.100, 0.920, "#bf{CMS Open Data}")
label.SetTextSize(0.030); label.DrawLatex(0.630, 0.920, "#sqrt{s} = 8 TeV, L_{int} = 11.6 fb^{-1}");

In [ ]:
c.Draw()

In [ ]:
report.Print()

RDataFrame can be easily turned into numpy ndarrays or pandas dataframes:

In [ ]:
#path = 'root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root'
path = './data/opendata_muons_skimmed.root'
df = ROOT.RDataFrame('Events', path)

npy = df.Filter('nMuon == 2')\
        .Filter('Muon_pt[0] != Muon_pt[1]')\
        .Define('Dimuon_mass', 'InvariantMass(Muon_pt, Muon_eta, Muon_phi, Muon_mass)')\
        .Range(10000)\
        .AsNumpy(['Dimuon_mass'])

In [ ]:
pdf = pd.DataFrame(npy)
print(pdf)

In [ ]:
plt.hist(npy['Dimuon_mass'], range=(70, 110), bins=20)
plt.xlabel('Dimoun mass in GeV');

### **awkward array**

In analysis we often have to work with data that can't be represented as a simple table (as if some branches of the tree are arrays of various size).

__[Awkward Arrays](https://github.com/scikit-hep/awkward-1.0)__ provides support for such general tree-like data structures, that are also contiguous in memory and operated upon with compiled, vectorized code like NumPy.

In [ ]:
import awkward1 as ak # when we were preparing this notebook there was the transition awkward (v0) -> awkward (v1)  happening, hence this weird naming

In [ ]:
array = ak.Array([
    [{"mu_m": 90, "jets_m": [1]}, 
     {"mu_m": 50, "jets_m": [1, 2]}, 
     {"mu_m": [30,50,70], "jets_m": [1, 2, 3]}],
    
    [],
    
    [{"mu_m": 40, "jets_m": [1, 2, 3, 4]}, 
     {"mu_m": [], "jets_m": [1, 2, 3, 4, 5]}]
])


In [ ]:
print(array["mu_m"])
print(array["jets_m"])

In [ ]:
output = np.square(array["jets_m", ..., 1:])
print(output)

### **uproot**

[uproot](https://github.com/scikit-hep/uproot4) (originally μproot, for "micro-Python ROOT") is a reader and a writer of the ROOT file format using only Python and Numpy. Unlike the standard C++ ROOT implementation, uproot is only an I/O library, primarily intended to stream data into machine learning libraries in Python. Unlike PyROOT and root_numpy, uproot does not depend on C++ ROOT. Instead, it uses Numpy to cast blocks of data from the ROOT file as Numpy arrays.

Uproot is a Python package; it is pip and conda-installable, and it only depends on other Python packages. Although it is similar in function to root_numpy and root_pandas, it does not compile into ROOT and therefore avoids issues in which the version used in compilation differs from the version encountered at runtime.

In [ ]:
import uproot4 as uproot # when we were preparing this notebook there was the transition uproot (v3) -> uproot (v4)  happening, hence this weird naming

file = uproot.open("http://scikit-hep.org/uproot3/examples/nesteddirs.root")
file

uproot.open returns a ROOTDirectory, which behaves like a Python dict; it has keys(), values(), and key-value access with square brackets.

In [ ]:
file.keys()

In [ ]:
file['one']

In [ ]:
file['one'].values()

TTrees are special objects in ROOT files: they contain most of the physics data. Uproot presents TTrees as subclasses of TTreeMethods.

In [ ]:
path = './data/opendata_muons_skimmed.root'

file = uproot.open(path)
file

In [ ]:
file.keys()

In [ ]:
file.classnames()

In [ ]:
tree = file["Events"]
tree.show()

In [ ]:
tree.keys()

In [ ]:
tree.items()

In [ ]:
tree.typenames()

In [ ]:
tree.keys(filter_name="Muon_*")

In [ ]:
tree.keys(filter_typename="float[]")

In [ ]:
tree.keys(filter_branch=lambda branch: not isinstance(branch.interpretation, uproot.AsJagged))

In [ ]:
tree["Muon_pt"].array()

In [ ]:
tree["Muon_pt"].array()[:20].tolist()

In [ ]:
tree["Muon_pt"].array(library="np")[:20]

In [ ]:
awkward_array = tree["Muon_pt"].array(library="ak")
numpy_array = tree["Muon_pt"].array(library="np")

In [ ]:
awkward_array[:20, 0]

In [ ]:
numpy_array[:20]

In [ ]:
tree["Muon_pt"].array(library="pd")

In [ ]:
plt.hist(uproot.open("./data/opendata_muons_skimmed.root:Events/nMuon"), bins=11, range=(-0.5, 10.5));

In [ ]:
# NumPy arrays in a dict
pv_numpy = tree.arrays(filter_name="Muon_*", library="np")
pv_numpy

In [ ]:
# Awkward record-array
pv_awkward = tree.arrays(filter_name="Muon_*", library="ak")
pv_awkward

In [ ]:
# Pandas DataFrame (as opposed to Series for a single array)
pv_pandas = tree.arrays(filter_name="Muon_*", library="pd")
pv_pandas

Uproot also has a limited ability to write ROOT files, including TTrees of flat data (non-jagged: single number per event), a variety of histogram types, and TObjString (for metadata).

## **Fitting**

### **iminuit**

[iminuit](https://iminuit.readthedocs.io/en/stable/) is a Jupyter-friendly Python frontend to the MINUIT2 C++ library (which is also used for fitting in ROOT).

It can be used as a general robust function minimisation method, but is most commonly used for likelihood fits of models to data, and to get model parameter error estimates from likelihood profile analysis.

In the next example it is show how it can be used to perform simple least squares fit

In [ ]:
# everything in iminuit is done through the Minuit object, so we import it
from iminuit import Minuit
from iminuit.util import describe, make_func_code
# we also need a cost function to fit and import the LeastSquares function
from iminuit.cost import LeastSquares
import matplotlib.pyplot as plt 
import math


In [ ]:
# our line model
def line(x, a, b):
    return a + x * b

# generate random toy data with random offsets in y
np.random.seed(1)
data_x = np.linspace(0, 1, 10)
data_yerr = 0.1 # could also be an array
data_y = line(data_x, 1, 2) + data_yerr * np.random.randn(len(data_x))

# draw toy data
plt.errorbar(data_x, data_y, data_yerr, fmt="o")
plt.xlim(-0.1, 1.1);

In [ ]:
# a simple least-squares cost function looks like this...
def custom_least_squares(a, b):
    ym = line(data_x, a, b)
    z = (data_y - ym) / data_yerr ** 2
    return np.sum(z ** 2)

# ...but instead of writing this by hand, it is better and
# more convenient to use the LeastSquares class shipped with iminuit
least_squares = LeastSquares(data_x, data_y, data_yerr, line)

m = Minuit(least_squares, a=0, b=0)  # we need starting values for a and b

m.migrad() # finds minimum of least_squares function
m.hesse()  # computes errors 

# draw data and fitted line
plt.errorbar(data_x, data_y, data_yerr, fmt="o")
plt.plot(data_x, line(data_x, *m.values.values()))

# print parameter values and uncertainty estimates
for p in m.parameters:
    print("{} = {:.3f} +/- {:.3f}".format(p, m.values[p], m.errors[p]))

In [ ]:
data = np.load('./data/pol_data_09_Dec_2020_14:10:54.npy')

In [ ]:
print(data[:10])

In [ ]:
def plot_pol_data(data, mod=None):
    
    fig = plt.figure(figsize=(10, 4))
    ax1 = fig.add_subplot(121)
    ax1.grid()
    ax2 = fig.add_subplot(122, polar=True)

    ax1.plot(data[:,0],data[:,1],'ro')
   
    ax2.plot(data[:,0],data[:,1])
    if mod is not None:
        x_model = np.linspace(0,2*math.pi,num=200)
        y_model = fit_func(x_model,mod['a'], mod['omega'], mod['phi'],mod['offset'])
        ax1.plot(x_model,y_model,'b')
    plt.show() 
plot_pol_data(data)

In [ ]:
class MyChi2:
    def __init__(self, model, x, y):
        self.model = model  # model predicts y for given x
        self.x = np.array(x)
        self.y = np.array(y)
        self.y_err = np.where(self.y > 0, np.sqrt(self.y),1)
        self.func_code = make_func_code(describe(self.model)[1:])

    def __call__(self, *par):  # we accept a variable number of model parameters
        ym = self.model(self.x, *par)
        self.y = np.where(self.y_err > 0, self.y,0)
        chi2 = np.sum(np.where(self.y > 0, (self.y-ym)**2/self.y_err**2,0))
        return chi2


In [ ]:
def fit_func(x, a, omega, phi, offset):
    return offset + a*np.power(np.cos(omega*x + phi),2)

In [ ]:
chi2 = MyChi2(fit_func, data[:,0], data[:,1])
m = Minuit(chi2, 
    limit_a=(100,4500),
    error_a=1.,
    limit_omega=(0,2),
    error_omega=0.001,
    limit_phi=(-1.6,1.6),
    error_phi=0.001,
    limit_offset=(0,3000),
    error_offset=1,
    pedantic=True,
    errordef=1)
m.migrad()

In [ ]:
plot_pol_data(data, m.values)

### **zfit**

The basic idea behind [zfit](https://github.com/zfit/zfit) is to offer a Python oriented alternative to the very successful RooFit library from the ROOT data analysis package that can integrate with the other packages that are part if the scientific Python ecosystem. Contrary to the monolithic approach of ROOT/RooFit, the aim of zfit is to be light and flexible enough to integrate with any state-of-art tools and to allow scalability going to larger datasets.

In [ ]:
import tensorflow as tf

import zfit
# Wrapper for some tensorflow functionality
from zfit import z
print("TensorFlow version:", tf.__version__)

In [ ]:
obs = zfit.Space('x', limits=(-10, 10))

In [ ]:
mu_true = 0
sigma_true = 1

data_np = np.random.normal(mu_true, sigma_true, size=10000)
data = zfit.data.Data.from_numpy(obs=obs, array=data_np)

In [ ]:
mu = zfit.Parameter("mu", 2.4, -1., 5., step_size=0.001)  # step_size is not mandatory but can be helpful
sigma = zfit.Parameter("sigma", 1.3, 0, 5., step_size=0.001)  # it should be around the estimated uncertainty

In [ ]:
gauss = zfit.pdf.Gauss(obs=obs, mu=mu, sigma=sigma)

In [ ]:
# Create the negative log likelihood

nll = zfit.loss.UnbinnedNLL(model=gauss, data=data)  # loss

# Load and instantiate a minimizer
minimizer = zfit.minimize.Minuit()
minimum = minimizer.minimize(loss=nll)

# Get the fitted values, again by run the variable graphs
params = minimum.params

print("mu={}".format(params[mu]['value']))
print("sigma={}".format(params[sigma]['value']))

In [ ]:
n_bins = 50
range_ = (-5,5)
_ = plt.hist(data_np, bins=n_bins, range=range_)
x = np.linspace(*range_, num=1000)
with gauss.set_norm_range(range_):
    pdf = zfit.run(gauss.pdf(x))
_ = plt.plot(x, data_np.shape[0] / n_bins * (range_[1] - range_[0]) * pdf)

### **pyhf**

The HistFactory p.d.f. template [__[CERN-OPEN-2012-016](https://cds.cern.ch/record/1456844)__] is per-se independent of its implementation in ROOT and sometimes, it’s useful to be able to run statistical analysis outside of ROOT, RooFit, RooStats framework.

[pyhf](https://github.com/scikit-hep/pyhf) is a pure-python implementation of that statistical model for multi-bin histogram-based analysis and its interval estimation is based on the asymptotic formulas of “Asymptotic formulae for likelihood-based tests of new physics” [__[arXiv:1007.1727](https://arxiv.org/abs/1007.1727)__]. The aim is also to support modern computational graph libraries such as PyTorch and TensorFlow in order to make use of features such as autodifferentiation and GPU acceleration.

In [ ]:
import pyhf
import pyhf.contrib.viz.brazil

pyhf.set_backend("numpy")
model = pyhf.simplemodels.hepdata_like(
    signal_data=[10.0], bkg_data=[50.0], bkg_uncerts=[7.0]
)
data = [55.0] + model.config.auxdata

poi_vals = np.linspace(0, 5, 41)
results = [
    pyhf.infer.hypotest(test_poi, data, model, qtilde=True, return_expected_set=True)
    for test_poi in poi_vals
]

fig, ax = plt.subplots()
fig.set_size_inches(7, 5)
ax.set_xlabel(r"$\mu$ (POI)")
ax.set_ylabel(r"$\mathrm{CL}_{s}$")
pyhf.contrib.viz.brazil.plot_results(ax, poi_vals, results)

### **hepstats**

The [hepstats](https://github.com/scikit-hep/hepstats) module includes modeling, hypotests and [sPlot](https://root.cern.ch/doc/v612/classRooStats_1_1SPlot.html) submodules.

The modeling submodule includes the Bayesian Block algorithm that can be used to improve the binning of histograms. The visual improvement can be dramatic, and more importantly, this algorithm produces histograms that accurately represent the underlying distribution while being robust to statistical fluctuations. Here is a small example of the algorithm applied on Laplacian sampled data, compared to a histogram of this sample with a fine binning.

In [ ]:
from hepstats.modeling import bayesian_blocks

data = np.random.laplace(size=10000)
blocks = bayesian_blocks(data)

plt.hist(data, bins=1000, label='Fine Binning', density=True, alpha=0.6)
plt.hist(data, bins=blocks, label='Bayesian Blocks', histtype='step', density=True, linewidth=2)
plt.legend(loc=2)

### **hepunits**

[hepunits](https://github.com/scikit-hep/hepunits) collects the most commonly used units and constants in the HEP System of Units, as derived from the basic units originally defined by the CLHEP project. The package is in agreement with the values in the 2020 Particle Data Group review.

In [ ]:
from hepunits.constants import c_light
from hepunits.units     import picosecond, micrometer
tau_Bs = 1.5 * picosecond    # a particle lifetime, say the Bs meson's
ctau_Bs = c_light * tau_Bs   # ctau of the particle, ~450 microns
print(ctau_Bs)                # result in HEP units, so mm
print(ctau_Bs / micrometer)

In [ ]:
# add two quantities with length units and get the result in meters
from hepunits import units as u
print((1 * u.meter + 5 * u.cm) / u.meter)
# the default result is, of course, in HEP units, so mm
print(1 * u.meter + 5 * u.cm)